In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasets/skoda.csv')
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Octavia,2017,10550,Manual,25250,Petrol,150,54.3,1.4
1,Citigo,2018,8200,Manual,1264,Petrol,145,67.3,1.0
2,Octavia,2019,15650,Automatic,6825,Diesel,145,67.3,2.0
3,Yeti Outdoor,2015,14000,Automatic,28431,Diesel,165,51.4,2.0
4,Superb,2019,18350,Manual,10912,Petrol,150,40.9,1.5
...,...,...,...,...,...,...,...,...,...
6262,Yeti,2014,11440,Semi-Auto,14569,Petrol,160,44.8,1.2
6263,Octavia,2014,10990,Semi-Auto,49999,Petrol,30,56.5,1.4
6264,Fabia,2017,9500,Semi-Auto,17131,Petrol,145,61.4,1.0
6265,Citigo,2016,5999,Manual,21747,Petrol,20,62.8,1.0


In [3]:
df['model'] = df['model'].str.strip()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6267 entries, 0 to 6266
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         6267 non-null   object 
 1   year          6267 non-null   int64  
 2   price         6267 non-null   int64  
 3   transmission  6267 non-null   object 
 4   mileage       6267 non-null   int64  
 5   fuelType      6267 non-null   object 
 6   tax           6267 non-null   int64  
 7   mpg           6267 non-null   float64
 8   engineSize    6267 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 440.8+ KB


In [5]:
df.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

## Importing ML Libraries

In [6]:
import numpy as np
import sklearn.metrics as m
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
# cross validation
from sklearn.model_selection import cross_val_score
# grid search
from sklearn.model_selection import GridSearchCV
# outlier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.compose import ColumnTransformer

## Feature Selection

In [7]:
X = df.drop(columns=['price'])
y = df['price']

In [8]:
X

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,Octavia,2017,Manual,25250,Petrol,150,54.3,1.4
1,Citigo,2018,Manual,1264,Petrol,145,67.3,1.0
2,Octavia,2019,Automatic,6825,Diesel,145,67.3,2.0
3,Yeti Outdoor,2015,Automatic,28431,Diesel,165,51.4,2.0
4,Superb,2019,Manual,10912,Petrol,150,40.9,1.5
...,...,...,...,...,...,...,...,...
6262,Yeti,2014,Semi-Auto,14569,Petrol,160,44.8,1.2
6263,Octavia,2014,Semi-Auto,49999,Petrol,30,56.5,1.4
6264,Fabia,2017,Semi-Auto,17131,Petrol,145,61.4,1.0
6265,Citigo,2016,Manual,21747,Petrol,20,62.8,1.0


## Column Extraction

In [9]:
bin_cols = ['transmission', 'fuelType']
num_cols = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
cat_cols = ['model']

## Preprocessing Pipeline

In [10]:
# create pipeline
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])
binary_pipeline = Pipeline([
    ('one_hot_encoder', OrdinalEncoder())
])
cat_pipeline = Pipeline([
    ('one_hot_encoder', OneHotEncoder(drop='first'))
])

# create column transformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('binary', binary_pipeline, bin_cols),
    ('cat', cat_pipeline, cat_cols)
])

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('std_scaler',
                                                  StandardScaler())]),
                                 ['year', 'mileage', 'tax', 'mpg',
                                  'engineSize']),
                                ('binary',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OrdinalEncoder())]),
                                 ['transmission', 'fuelType']),
                                ('cat',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OneHotEncoder(drop='first'))]),
                                 ['model'])])

## Outlier Removal
- this help to remove the outliers from the dataset

In [11]:
outlier_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('outlier', LocalOutlierFactor(n_neighbors=20, contamination=0.1))
])

outlier_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('outlier', LocalOutlierFactor(contamination=0.1))])

In [12]:
yhat = outlier_pipeline.fit_predict(X)
X = X[yhat==1].copy()
y = y[yhat==1].copy()

In [13]:
df = pd.concat([X, y], axis=1)

In [14]:
X# cleane data

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,Octavia,2017,Manual,25250,Petrol,150,54.3,1.4
1,Citigo,2018,Manual,1264,Petrol,145,67.3,1.0
2,Octavia,2019,Automatic,6825,Diesel,145,67.3,2.0
3,Yeti Outdoor,2015,Automatic,28431,Diesel,165,51.4,2.0
4,Superb,2019,Manual,10912,Petrol,150,40.9,1.5
...,...,...,...,...,...,...,...,...
6261,Octavia,2018,Manual,30805,Petrol,145,57.7,1.5
6263,Octavia,2014,Semi-Auto,49999,Petrol,30,56.5,1.4
6264,Fabia,2017,Semi-Auto,17131,Petrol,145,61.4,1.0
6265,Citigo,2016,Manual,21747,Petrol,20,62.8,1.0


## Model Pipeline

In [15]:
pipeline1 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])
pipeline1

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', DecisionTreeRegressor())])

In [16]:
pipeline2 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])
pipeline2

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', RandomForestRegressor())])

In [17]:
pipeline3 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])
pipeline3

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', LinearRegression())])

In [18]:
pipeline4 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', SVR())
])
pipeline4

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', SVR())])

## Training Model and Evaluation

In [19]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model1 = pipeline1.fit(train_X, train_y)
yhat = model1.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Decision Tree Regressor")
print(f'MAE: {mae} - tells us how far off our predictions are on average.')
print(f'MSE: {mse} - tells us how far off our predictions are on average squared.')
print(f'RMSE: {rmse} - tells us how far off our predictions are on average squared root.')
print(f'R2: {r2} - tells us how much of the variance in the target variable is explained by the model.')

Decision Tree Regressor
MAE: 1299.9980791962175 - tells us how far off our predictions are on average.
MSE: 7413534.043415091 - tells us how far off our predictions are on average squared.
RMSE: 2722.7805720283613 - tells us how far off our predictions are on average squared root.
R2: 0.8362871893046877 - tells us how much of the variance in the target variable is explained by the model.


In [20]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model2 = pipeline2.fit(train_X, train_y)
yhat = model2.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Random Forest Regressor")
print(f'MAE: {mae} - tells us how far off our predictions are on average.')
print(f'MSE: {mse} - tells us how far off our predictions are on average squared.')
print(f'RMSE: {rmse} - tells us how far off our predictions are on average squared root.')
print(f'R2: {r2} - tells us how much of the variance in the target variable is explained by the model.')

Random Forest Regressor
MAE: 1072.6259676303307 - tells us how far off our predictions are on average.
MSE: 6207104.128841669 - tells us how far off our predictions are on average squared.
RMSE: 2491.406054588788 - tells us how far off our predictions are on average squared root.
R2: 0.8629287385395161 - tells us how much of the variance in the target variable is explained by the model.


In [21]:
model3 = pipeline3.fit(train_X, train_y)
yhat = model3.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Linear Regression")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

Linear Regression
MAE: 1467.020758133716
MSE: 7721579.280020418
RMSE: 2778.772981015257
R2: 0.8294846372140616


In [22]:
model4 = pipeline4.fit(train_X, train_y)
yhat = model4.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Support Vector Regression")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

Support Vector Regression
MAE: 4720.938992284834
MSE: 43500379.557699
RMSE: 6595.481753268597
R2: 0.039382653131620926


In [23]:
df['dtree_pred'] = model1.predict(df.drop(columns=['price']))
df['rf_pred'] = model2.predict(df.drop(columns=['price']))
df['lr_pred'] = model3.predict(df.drop(columns=['price']))
df['svr_pred'] = model4.predict(df.drop(columns=['price']))

## Visualization

In [24]:
# histogram
fig = px.histogram(df, x='price', nbins=100, title='Price Distribution')
fig.show()

## Saving the model

In [25]:
import joblib

In [26]:
joblib.dump(model1, 'skoda/dtree_model.pkl')
joblib.dump(model2, 'skoda/rf_model.pkl')
joblib.dump(model3, 'skoda/lr_model.pkl')
joblib.dump(model4, 'skoda/svr_model.pkl')

['skoda/svr_model.pkl']

In [27]:
X.fuelType.unique()

array(['Petrol', 'Diesel', 'Other'], dtype=object)

In [28]:
X.model.unique()

array(['Octavia', 'Citigo', 'Yeti Outdoor', 'Superb', 'Kodiaq', 'Rapid',
       'Karoq', 'Fabia', 'Yeti', 'Scala', 'Roomster', 'Kamiq'],
      dtype=object)

In [29]:
X.transmission.unique()

array(['Manual', 'Automatic', 'Semi-Auto'], dtype=object)

In [30]:
X

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,Octavia,2017,Manual,25250,Petrol,150,54.3,1.4
1,Citigo,2018,Manual,1264,Petrol,145,67.3,1.0
2,Octavia,2019,Automatic,6825,Diesel,145,67.3,2.0
3,Yeti Outdoor,2015,Automatic,28431,Diesel,165,51.4,2.0
4,Superb,2019,Manual,10912,Petrol,150,40.9,1.5
...,...,...,...,...,...,...,...,...
6261,Octavia,2018,Manual,30805,Petrol,145,57.7,1.5
6263,Octavia,2014,Semi-Auto,49999,Petrol,30,56.5,1.4
6264,Fabia,2017,Semi-Auto,17131,Petrol,145,61.4,1.0
6265,Citigo,2016,Manual,21747,Petrol,20,62.8,1.0
